In [7]:
import os
import numpy as np
import robotic as ry
import manipulation as manip
import matplotlib.pyplot as plt

from utils import sample_arena

In [8]:
gripper = "l_gripper"
palm = "l_palm"
boxes = [f"box{i}" for i in range(1, 4)]
table = "table"

In [9]:
def scene_generator():
    C = ry.Config()
    C.addFile(ry.raiPath("scenarios/pandaSingleWithTopCamera.g"))

    for i in range(3):
        # Frame name
        box_name = f"box{i+1}"

        # Box color
        color = [0, 0, 0]
        color[i] = 1

        # Position
        position_val1 = 0.2 * i + .3

        C.addFrame(box_name) \
            .setPosition([position_val1, 0.05, 0.72]) \
            .setShape(ry.ST.ssBox, size=[0.04, 0.04, 0.12, 0.001]) \
            .setColor(color) \
            .setContact(1) \
            .setMass(.1)

    C.delFrame("panda_collCameraWrist")

    A = .4
    B = .35
    OFFSET = [0, .2]

    for i in range(1,4):        
        x_y_pos = sample_arena(A, B, OFFSET)[:2]
        x_y_pos.append(.72)
        C.getFrame(f"box{i}").setPosition(x_y_pos)

    return C

In [10]:
def path_generator(C: ry.Config, target_position: np.ndarray):
	boxes = ["box1", "box2", "box3"]
	
	# Pick place waypoints
	M = manip.ManipulationModelling()
	M.setup_pick_and_place_sequence(C, gripper, table, boxes)

	# Box 1
	grasp_direction = np.random.choice(["x", "y"])
	M.grasp_box(1., gripper, "box1", palm, grasp_direction)
	M.place_box(2., "box1", table, palm, "z")
	M.target_relative_xy_position(2., "box1", table, target_position)

	# Box 2
	grasp_direction = np.random.choice(["x", "y"])
	M.grasp_box(3., gripper, "box2", palm, grasp_direction)
	M.place_box(4., "box2", table, palm, "z")
	M.set_relative_distance(4., "box1", "box2", .05)

	# Box 3
	grasp_direction = np.random.choice(["x", "y"])
	place_direction = "x" if grasp_direction == "y" else "y"
	M.grasp_box(5., gripper, "box3", palm, grasp_direction)
	M.place_box(6., "box3", "box1", palm, place_direction)
	# M.set_relative_distance(6., "box3", "box2", .0)

	M.solve(verbose=0)
	if not M.feasible: return False, []
	
	# Sub motions
	paths = []
	for i, box in enumerate(boxes):
		idx = i*2
		M1 = M.sub_motion(idx, accumulated_collisions=True)
		M1.retract([.0, .2], gripper)
		M1.approach([.8, 1.], gripper)
		path = M1.solve(verbose=0)
		if not M1.feasible:
			return False, []
		paths.append(path)

		M2 = M.sub_motion(idx+1, accumulated_collisions=True)
		path = M2.solve(verbose=0)
		if not M2.feasible:
			return False, []
		paths.append(path)

	return True, paths

In [ ]:
def capture_data(C: ry.Config, paths: np.ndarray, ep_idx: int):
    C.view()
    C.view_setCamera(C.getFrame("topCamera"))

    im_path = f"./bridge_episodes/{ep_idx}/images"
    if not os.path.exists(im_path):
        os.makedirs(im_path)
    
    timestep_count = 0
    grasping_value = 0
    states = []
    actions = []
    C.setJointState(paths[0][0])
    for i, path in enumerate(paths):
        for j in range(1, len(path)):

            timestep_count += 1

            C.view()
            image = C.view_getRgb()
            plt.imsave(f"{im_path}/{timestep_count}.png", image)
            
            state = path[j-1]
            action = path[j]

            states.append([*state, grasping_value])
            actions.append([*action, grasping_value])

            C.setJointState(path[j])

        if i%2:
            C.attach(table, boxes[i//2])
            grasping_value = 0
        else:
            C.attach(gripper, boxes[i//2])
            grasping_value = 1

    states = np.array(states)
    actions = np.array(actions)
    np.save(f"./bridge_episodes/{ep_idx}/state", states)
    np.save(f"./bridge_episodes/{ep_idx}/actions", actions)

In [12]:
MAX_EPISODE_COUNT = 500
episode_count = 0
for i in range(MAX_EPISODE_COUNT):
	attempt_count = 10
	midpoint = [-0.105, 0.4, 0.745]

	C = scene_generator()

	for l in range(attempt_count):
		housePosition = [midpoint[0] + np.random.random()*.2 -.1, midpoint[1] + np.random.random()*.2]
		success, paths = path_generator(C, housePosition)
		if success:
			episode_count += 1
			print(f"Saving_episode {episode_count}... ", end="")
			capture_data(C, paths, episode_count)
			print("Done!")
			break
	del C

0
1
Saving_episode 1... Done!
0
Saving_episode 2... Done!
0
1
2
3
4
5
6
7
8
9
0
Saving_episode 3... Done!
0
Saving_episode 4... Done!
0
Saving_episode 5... Done!
0
1
2
Saving_episode 6... Done!
0
Saving_episode 7... Done!
0
Saving_episode 8... Done!
0
1
Saving_episode 9... Done!
0
Saving_episode 10... Done!
0
1
2
3
4
5
6
7
8
9
0
Saving_episode 11... Done!
0
1
2
3
4
5
6
7
8
9
0
1
2
3
4
5
6
7
8
9
0
Saving_episode 12... Done!
0
1
Saving_episode 13... Done!
0
1
2
3
4
5
6
7
8
9
0
1
2
3
Saving_episode 14... Done!
0
Saving_episode 15... Done!
0
Saving_episode 16... Done!
0
Saving_episode 17... Done!
0
1
2
Saving_episode 18... Done!
0
1
2
3
Saving_episode 19... Done!
0
1
2
3
4
5
6
7
8
9
0
1
Saving_episode 20... Done!
0
Saving_episode 21... Done!
0
Saving_episode 22... Done!
0
1
Saving_episode 23... Done!
0
Saving_episode 24... Done!
0
Saving_episode 25... Done!
0
Saving_episode 26... Done!
0
Saving_episode 27... Done!
0
Saving_episode 28... Done!
0
Saving_episode 29... Done!
0
Saving_episode 3